In [1]:
path='C:\\Users\\umairansari\\Desktop\\comp data\\analytics vidya\\janata hack'
import os
os.chdir(path)
print(os.getcwd())
import pandas as pd
pd.set_option('use_inf_as_na', True)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
os.listdir()
# from bayes_opt import BayesianOptimization
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier,StackingRegressor
from sklearn.neighbors import KNeighborsClassifier

#from imblearn.over_sampling import SMOTE 
import warnings
#warnings.filterwarnings('ignore')
import gc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier,ExtraTreeRegressor,DecisionTreeRegressor
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBRFClassifier,XGBRFRegressor
from sklearn.linear_model import PassiveAggressiveClassifier,RidgeClassifierCV,SGDClassifier,ElasticNetCV
from sklearn.svm import SVC

C:\Users\umairansari\Desktop\comp data\analytics vidya\janata hack


In [2]:
trained=pd.read_csv("train_jqd04QH.csv")
test=pd.read_csv("test_KaymcHn.csv")
trained["flag"]="train"
test["flag"]="test"

In [3]:
trained.describe()

,enrollee_id,city_development_index,training_hours,target
count,18359.000000,18359.000000,18359.000000,18359.000000
mean,16729.360096,0.847140,65.899014,0.132088
std,9643.749725,0.110189,60.885300,0.338595
min,1.000000,0.448000,1.000000,0.000000
25%,8378.500000,0.796000,23.000000,0.000000
50%,16706.000000,0.910000,47.000000,0.000000
75%,25148.500000,0.920000,89.000000,0.000000
max,33380.000000,0.949000,336.000000,1.000000


In [4]:
train=pd.concat([trained,test])

C:\Users\umairansari\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [5]:
# import missingno as msno
# msno.bar(train)

In [6]:
for i in train.columns:
    x=train[i].isna().sum()
    print("missing values in {} are {}%".format(i,(x/train.shape[0])*100))

missing values in city are 0.0%
missing values in city_development_index are 0.0%
missing values in company_size are 26.452965847813058%
missing values in company_type are 28.067705212702215%
missing values in education_level are 2.5524266027561415%
missing values in enrolled_university are 1.860395446375075%
missing values in enrollee_id are 0.0%
missing values in experience are 0.3085680047932894%
missing values in flag are 0.0%
missing values in gender are 22.426602756141403%
missing values in last_new_job are 2.0101857399640504%
missing values in major_discipline are 15.67106051527861%
missing values in relevent_experience are 0.0%
missing values in target are 45.0%
missing values in training_hours are 0.0%


In [7]:
#unique_values
for i in train.columns:
    print("unique values in {} are {} ".format(i,len(train[i].unique())))

unique values in city are 123 
unique values in city_development_index are 93 
unique values in company_size are 9 
unique values in company_type are 7 
unique values in education_level are 6 
unique values in enrolled_university are 4 
unique values in enrollee_id are 33380 
unique values in experience are 23 
unique values in flag are 2 
unique values in gender are 4 
unique values in last_new_job are 7 
unique values in major_discipline are 7 
unique values in relevent_experience are 2 
unique values in target are 3 
unique values in training_hours are 241 


# DATA CLEANING

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33380 entries, 0 to 15020
Data columns (total 15 columns):
city                      33380 non-null object
city_development_index    33380 non-null float64
company_size              24550 non-null object
company_type              24011 non-null object
education_level           32528 non-null object
enrolled_university       32759 non-null object
enrollee_id               33380 non-null int64
experience                33277 non-null object
flag                      33380 non-null object
gender                    25894 non-null object
last_new_job              32709 non-null object
major_discipline          28149 non-null object
relevent_experience       33380 non-null object
target                    18359 non-null float64
training_hours            33380 non-null int64
dtypes: float64(2), int64(2), object(11)
memory usage: 4.1+ MB


In [9]:
train.isna().sum()

city                          0
city_development_index        0
company_size               8830
company_type               9369
education_level             852
enrolled_university         621
enrollee_id                   0
experience                  103
flag                          0
gender                     7486
last_new_job                671
major_discipline           5231
relevent_experience           0
target                    15021
training_hours                0
dtype: int64

In [10]:
#---------------------------------------------------------------------------------
train["city"]=train["city"].str.replace("city_","")
train["city"]=pd.to_numeric(train["city"],errors='coerce')

#---------------------------------------------------------------------------------

train["company_type"]=train["company_type"].fillna("Other1")
train["company_type"]=train["company_type"].map({'Pvt Ltd':0, 'Funded Startup':1, 'Public Sector':2, 'Other1':6,
                                                 'Early Stage Startup':3, 'NGO':4, 'Other':5})

#---------------------------------------------------------------------------------
train["gender"]=train["gender"].fillna("other")
train["gender"]=train["gender"].map({'Male':1,'other':3,'Female':0,'Other':2})


#---------------------------------------------------------------------------------

train["company_size"]=train["company_size"].str.replace("/","0")
train["company_size"]=train["company_size"].str.replace("<","5-")
train["company_size"]=train["company_size"].fillna("unknown")

#---------------------------------------------------------------------------------
train["relevent_experience"]=train["relevent_experience"].map({'Has relevent experience':1,'No relevent experience':0})


#---------------------------------------------------------------------------------
# cleaning education level
train["education_level"]=train["education_level"].fillna("Other_Education_or_not_educated")
train["education_level"]=train["education_level"].map({'Graduate':3, 'Masters':4, 'High School':2, 'Phd':5,
       'Other_Education_or_not_educated':0, 'Primary School':1})

#---------------------------------------------------------------------------------
#filling na  of major with no major because people who have not been graduated are not eligible for majors
#people who study less than garduate are not eligible for majors
trained_m=train[train["education_level"]<3]
print(trained_m.shape)
print(train["major_discipline"].isna().sum())
train["major_discipline"]=train["major_discipline"].fillna("No Major")

#---------------------------------------------------------------------------------

#cleaning experience level and filling na
train["experience"]=train["experience"].str.replace(">20","25")
train["experience"]=train["experience"].str.replace("<1","0")
train["experience"]=pd.to_numeric(train["experience"],errors='coerce')
train["experience"]=train["experience"].fillna(0)

#---------------------------------------------------------------------------------
train["enrolled_university"]=train["enrolled_university"].fillna("no_enrollment")
train["enrolled_university"]=train["enrolled_university"].map({'no_enrollment':0,
                                                               'Full time course':1,
                                                               'Part time course':2})

#---------------------------------------------------------------------------------
train["last_new_job"]=train["last_new_job"].str.replace("never","0")

train["last_new_job"]=train["last_new_job"].str.replace(">4","5")
train["last_new_job"]=train["last_new_job"].fillna("0")
train["last_new_job"]=pd.to_numeric(train["last_new_job"],errors='coerce')




(5190, 15)
5231


In [11]:
# train["company_size"]=train["company_size"].map({'100-500':2, '5-10':0, '50-99':1, 'unknown':8, '5000-9999':5, '10000+':7,
#        '1000-4999':4, '500-999':3, '10049':6})

# train["major_discipline"]=train["major_discipline"].map({'STEM':2, 'Other':5, 'No Major':0, 'Business Degree':3, 'Arts':4,
#        'Humanities':1})

# SOME EDA

In [12]:

# #checking the target column
# plt.figure(figsize=(10,6))
# plt.title("Target Column values count 1 are {} in number and 0 are {} in number"
#           .format(train[train["target"]==1].shape[0],train[train["target"]==0].shape[0]))
# train["target"].plot(kind='hist')
# plt.xlabel("frequency")
# plt.ylabel("target value")
# plt.xticks(ticks=[0,.5,1])


# #let see how target 0 columns depends on other columns
# target0=train[train["target"]==0]
# sns.pairplot(target0)



# #let see how target 0 columns depends on other columns
# target1=train[train["target"]==1]
# sns.pairplot(target1)



# plt.figure(figsize=(10,8))
# sns.scatterplot(y=train["training_hours"],x=train["experience"],hue=train["target"])


# #plotting the plot for 1 target
# data=train["experience"][train["target"]==1].value_counts()
# data=pd.DataFrame({"name":data.index,"values":data.values})
# plt.figure(figsize=(10,6))
# sns.barplot(data["name"],data["values"])



# #plotting the plot for 0 target
# data=train["experience"][train["target"]==0].value_counts()
# data=pd.DataFrame({"name":data.index,"values":data.values})
# plt.figure(figsize=(10,6))
# sns.barplot(data["name"],data["values"])


# plt.figure(figsize=(10,8))
# plt.title("Experience column counts")
# sns.distplot(train["experience"])
# plt.xticks(ticks=np.arange(0,26))



# plt.figure(figsize=(10,8))
# plt.title("Experience vs Training hours vs Target(People who have more training hours are more expecting for job change)")
# sns.barplot(train["experience"],train["training_hours"],hue=train["target"])
# #plt.xticks(ticks=np.arange(0,26))

# LETS STANDARDIZE THE DATA


In [13]:
train.columns

Index(['city', 'city_development_index', 'company_size', 'company_type',
       'education_level', 'enrolled_university', 'enrollee_id', 'experience',
       'flag', 'gender', 'last_new_job', 'major_discipline',
       'relevent_experience', 'target', 'training_hours'],
      dtype='object')

In [14]:
#---------------------------------------------------------------------------------
newdata=pd.get_dummies(data=train,columns=['company_size','major_discipline'],prefix="dum")
#---------------------------------------------------------------------------------
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
mms=MinMaxScaler()
ssc=StandardScaler()
rs=RobustScaler()
# newdata['city'] = ssc.fit_transform(newdata['city'].values.reshape(-1,1))
newdata['city'] = ssc.fit_transform(newdata['city'].values.reshape(-1,1))
#newdata['experience'] = mms.fit_transform(newdata['experience'].values.reshape(-1,1))
newdata["engineered_exp_training"]=round(newdata["training_hours"]/train["experience"],2)
newdata["engineered_exp_training1"]=round(newdata["training_hours"]*train["experience"],2)
newdata["engineered_exp_training"]=newdata["engineered_exp_training"].fillna(0)
#saperating our data
print(trained.shape)

idx=['city', 'city_development_index', 'experience', 'last_new_job' ,'training_hours']
for df in[newdata]:
    df['sum'] = round(df[idx].sum(axis=1),2)  
    df['min'] = round(df[idx].min(axis=1),2)
    df['max'] = round(df[idx].max(axis=1),2)
    df['mean'] = round(df[idx].mean(axis=1),2)
    df['std'] = round(df[idx].std(axis=1),2)
    df['skew'] = round(df[idx].skew(axis=1),2)
    df['kurt'] = round(df[idx].kurtosis(axis=1),2)
    df['med'] = round(df[idx].median(axis=1),2)

trained=newdata[newdata["flag"]=='train']
test=newdata[newdata["flag"]=='test']
print(trained.shape,test.shape)

    
# #saperating our data
# print(trained.shape)
# trained=newdata[newdata["flag"]=='train']
# test=newdata[newdata["flag"]=='test']
# print(trained.shape,test.shape)

(18359, 15)
(18359, 38) (15021, 38)


# MODELLING

In [15]:
#splitting the data into test train 
training,validation=train_test_split(trained,test_size=.30,random_state=42)
training.shape,validation.shape


# full_train=train[train["flag"]=="train"]
# full_train_x=full_train.drop(["target","flag","enrollee_id"],axis=1)
# full_train_y=full_train["target"]

full_train=newdata[newdata["flag"]=="train"]
full_train_x=full_train.drop(["target","flag","enrollee_id"],axis=1)
full_train_y=full_train["target"]
print(full_train_x.shape,full_train_y.shape)
# #---------------------smote
# sm = SMOTE(random_state = 2) 
# x_train_smote_full, y_train_smote_full = sm.fit_sample(full_train_x, full_train_y.ravel()) 



#######################################################Artificially oversampling using smote 
#saperating x and y
# training_x=x_train_smote_full.drop(["target","flag","enrollee_id"],axis=1)
# training_y=training["target"]
# validation_x=validation.drop(["target","flag","enrollee_id"],axis=1)
# validation_y=validation["target"]

#--------------------------------------------------------------------
test_final=test.drop(["target","flag","enrollee_id"],axis=1)

#saperating x and y
training_x=training.drop(["target","flag","enrollee_id"],axis=1)
training_y=training["target"]
validation_x=validation.drop(["target","flag","enrollee_id"],axis=1)
validation_y=validation["target"]

(18359, 35) (18359,)


In [16]:
test_final.shape

(15021, 35)

In [17]:
lrc=LogisticRegression(class_weight='balanced')
# gbc=GradientBoostingClassifier()
# rfc=RandomForestClassifier(class_weight='balanced')
# etc=ExtraTreeClassifier(class_weight='balanced')
# dtc=DecisionTreeClassifier(class_weight='balanced')
# abc=AdaBoostClassifier()
# svc=SVC(class_weight='balanced')
pac=PassiveAggressiveClassifier(class_weight='balanced')
rc=RidgeClassifierCV(class_weight='balanced')
sgc=SGDClassifier(class_weight='balanced')
enc=ElasticNetCV()
gnbc=GaussianNB()
knc=KNeighborsClassifier()

In [18]:
pac.fit(training_x,training_y)
rc.fit(training_x,training_y)
sgc.fit(training_x,training_y)
enc.fit(training_x,training_y)
lrc.fit(training_x,training_y)
gnbc.fit(training_x,training_y)
knc.fit(training_x,training_y)

# gbc.fit(training_x,training_y)
# rfc.fit(training_x,training_y)
# etc.fit(training_x,training_y)
# dtc.fit(training_x,training_y)
# abc.fit(training_x,training_y)
# svc.fit(training_x,training_y)


C:\Users\umairansari\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6139898024637205, tolerance: 0.11653307392996097
  tol, rng, random, positive)
C:\Users\umairansari\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.35929573279418037, tolerance: 0.11609082774049216
  tol, rng, random, positive)
C:\Users\umairansari\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.66299688974118, tolerance: 0.11609082774049216
  tol, rng, random, positive)
C:\Users\umairansari\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Obje

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [19]:
#prediction
pr_lrc=lrc.predict(validation_x)
pr_rc=rc.predict(validation_x)

pr_sgc=sgc.predict(validation_x)
pr_enc=enc.predict(validation_x)
pr_pac=pac.predict(validation_x)
pr_gnbc=gnbc.predict(validation_x)
pr_knn=knc.predict(validation_x)

# pr_gbc=gbc.predict(validation_x)
# pr_rfc=rfc.predict(validation_x)
# pr_etc=etc.predict(validation_x)
# pr_dtc=dtc.predict(validation_x)
# pr_abc=abc.predict(validation_x)

#predicted prob
proba_lrc=lrc.predict(validation_x)
# proba_gbc=gbc.predict(validation_x)
# proba_rfc=rfc.predict(validation_x)
# proba_etc=etc.predict(validation_x)
# proba_dtc=dtc.predict(validation_x)
# proba_abc=abc.predict(validation_x)

In [20]:
# from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,auc,roc_auc_score
# print(classification_report(pr_lrc,validation_y))
# print(classification_report(pr_gbc,validation_y))
# print(classification_report(pr_rfc,validation_y))
# print(classification_report(pr_etc,validation_y))
# print(classification_report(pr_dtc,validation_y))
# print(classification_report(pr_abc,validation_y))
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_auc_score
print(roc_auc_score(pr_lrc,validation_y))
print(roc_auc_score(pr_rc,validation_y))
print(roc_auc_score(pr_sgc,validation_y))
print(roc_auc_score(pr_pac,validation_y))
#print(roc_auc_score(pr_enc,validation_y))
print(roc_auc_score(pr_gnbc,validation_y))
print(roc_auc_score(pr_knn,validation_y))

# print(roc_auc_score(pr_gbc,validation_y))
# print(roc_auc_score(pr_rfc,validation_y))
# print(roc_auc_score(pr_etc,validation_y))
# print(roc_auc_score(pr_dtc,validation_y))
# print(roc_auc_score(pr_abc,validation_y))

0.543000573163167
0.5585282985993476
0.5492113291605654
0.5577159163706249
0.5549130245027745
0.5300541031413077


In [21]:
full_train_x["city"]

0        1.446127
1       -0.008823
2       -1.485818
3       -0.427672
4        0.365937
           ...   
18354    0.432071
18355    1.688618
18356    0.674562
18357   -0.185181
18358    0.476160
Name: city, Length: 18359, dtype: float64

In [22]:
# tn1, fp1, fn1, tp1 = confusion_matrix(pr_lrc,validation_y).ravel()
# # =confusion_matrix(pr_lrc,validation_y).ravel()
# tn2, fp2, fn2, tp2=confusion_matrix(pr_gbc,validation_y).ravel()
# tn3, fp3, fn3, tp3=confusion_matrix(pr_rfc,validation_y).ravel()
# tn4, fp4, fn4, tp4=confusion_matrix(pr_etc,validation_y).ravel()
# tn5, fp5, fn5, tp5=confusion_matrix(pr_dtc,validation_y).ravel()
# tn6, fp6, fn6, tp6=confusion_matrix(pr_abc,validation_y).ravel()

# print(tn1, fp1, fn1, tp1)
# print(tn2, fp2, fn2, tp2)
# print(tn3, fp3, fn3, tp3)
# print(tn4, fp4, fn4, tp4)
# print(tn5, fp5, fn5, tp5)
# print(tn6, fp6, fn6, tp6)


In [23]:
# (y_train_smote_full==0).sum()

In [24]:
#function to get variable importances
def plot_graph(a):
    plt.figure(figsize=(20,12))
    chart=sns.barplot(x=a["name_of_column"],y=a["feature_importance"])
    chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
def feature_importance_plot_rev(dataframe,classifier,limit_of_the_importance):
    x=list(zip(dataframe.columns,classifier.feature_importances_))
    xx=pd.DataFrame(x)
    xx.columns=["name_of_column","feature_importance"]
    xx=xx.sort_values("feature_importance",ascending=False)
    xx=xx[xx["feature_importance"]>limit_of_the_importance]
    plot_graph(xx)
    return xx

In [25]:
# feature_importance_plot_rev(training_x,rfc,0)

# feature_importance_plot_rev(training_x,dtc,0)

# feature_importance_plot_rev(training_x,etc,0)

# feature_importance_plot_rev(training_x,gbc,0)


#abc is giving good results
#feature_importance_plot_rev(training_x,abc,0)

# abc.fit(full_train_x,full_train_y)
# test_final=test.drop(["target","flag","enrollee_id"],axis=1)
# predicted_final_abc=abc.predict(test_final)

# #-------------------rfc predicted
# rfc.fit(x_train_smote_full,y_train_smote_full)
# test_final=test.drop(["target","flag","enrollee_id"],axis=1)
# predicted_final_rfc=rfc.predict(test_final)
# prediction=pd.DataFrame({'enrollee_id':test["enrollee_id"],'target':predicted_final_rfc})
# prediction.to_csv('predicted1_rfc_bal_ssc_smote.csv',index=False)


# #-----------------etc predicted
# etc.fit(x_train_smote_full,y_train_smote_full)
# test_final=test.drop(["target","flag","enrollee_id"],axis=1)
# predicted_final_etc=etc.predict(test_final)
# prediction=pd.DataFrame({'enrollee_id':test["enrollee_id"],'target':predicted_final_etc})
# prediction.to_csv('predicted1_etc_bal_ssc_smote.csv',index=False)


# # # #
# #----------------dtc precicted
# dtc.fit(x_train_smote_full,y_train_smote_full)
# test_final=test.drop(["target","flag","enrollee_id"],axis=1)
# predicted_final_dtc=dtc.predict(test_final)
# prediction=pd.DataFrame({'enrollee_id':test["enrollee_id"],'target':predicted_final_dtc})
# prediction.to_csv('predicted1_dtc_bal_ssc_smote.csv',index=False)

# #------------------svc

# svc.fit(x_train_smote_full,y_train_smote_full)
# test_final=test.drop(["target","flag","enrollee_id"],axis=1)
# predicted_final_svc=svc.predict(test_final)
# prediction=pd.DataFrame({'enrollee_id':test["enrollee_id"],'target':predicted_final_svc})
# prediction.to_csv('predicted1_svc_ssc.smote.csv',index=False)


#----------------------------------lgc
# selected_data=full_train_x[['training_hours', 'experience', 'city_development_index', 'city', 'last_new_job',
#                            'education_level', 'company_type', 'gender', 'enrolled_university', 'relevent_experience',
#                            'dum_unknown', 'dum_50-99', 'dum_STEM','dum_100-500', 'dum_10000+', 'dum_1000-4999']]
# selected_data_test=test[['training_hours', 'experience', 'city_development_index', 'city', 'last_new_job',
#                            'education_level', 'company_type', 'gender', 'enrolled_university', 'relevent_experience',
#                            'dum_unknown', 'dum_50-99', 'dum_STEM','dum_100-500', 'dum_10000+', 'dum_1000-4999']]

#--------------passive REGRESSION
#not working
pac.fit(full_train_x,full_train_y)
predicted_final_pac=pac.predict(test_final)
#print(classification_report(predicted_final_lrc,validation_y))
prediction=pd.DataFrame({'enrollee_id':test["enrollee_id"],'target':predicted_final_pac})
prediction.to_csv('predicted1_pac_bal_standardscaler.csv',index=False)
#--------------SGC
sgc.fit(full_train_x,full_train_y)
predicted_final_sgc=sgc.predict(test_final)
#print(classification_report(predicted_final_lrc,validation_y))
prediction=pd.DataFrame({'enrollee_id':test["enrollee_id"],'target':predicted_final_sgc})
prediction.to_csv('predicted1_sgc_bal_standardscaler.csv',index=False)


#--------------LOGISTIC REGRESSION
lrc.fit(full_train_x,full_train_y)
predicted_final_lrc=lrc.predict(test_final)
#print(classification_report(predicted_final_lrc,validation_y))
prediction=pd.DataFrame({'enrollee_id':test["enrollee_id"],'target':predicted_final_lrc})
prediction.to_csv('predicted1_lrc_bal_standardscaler.csv',index=False)
#--------------elastic REGRESSION
#done-working
enc.fit(full_train_x,full_train_y)
predicted_final_enc=enc.predict(test_final)
#print(classification_report(predicted_final_lrc,validation_y))
prediction=pd.DataFrame({'enrollee_id':test["enrollee_id"],'target':predicted_final_enc})
prediction.to_csv('predicted1_enc_bal_standardscaler.csv',index=False)

#--------------ridge
#done working
rc.fit(full_train_x,full_train_y)
predicted_final_rc=rc.predict(test_final)
#print(classification_report(predicted_final_lrc,validation_y))
prediction=pd.DataFrame({'enrollee_id':test["enrollee_id"],'target':predicted_final_rc})
prediction.to_csv('predicted1_rc_bal_standardscaler.csv',index=False)

#--------------NB
#done not working
gnbc.fit(full_train_x,full_train_y)
predicted_final_gnbc=gnbc.predict(test_final)
#print(classification_report(predicted_final_lrc,validation_y))
prediction=pd.DataFrame({'enrollee_id':test["enrollee_id"],'target':predicted_final_gnbc})
prediction.to_csv('predicted1_gnbc_bal_standardscaler.csv',index=False)


#---------------knn
#done not working
knc.fit(full_train_x,full_train_y)
predicted_final_knc=knc.predict(test_final)
#print(classification_report(predicted_final_lrc,validation_y))
prediction=pd.DataFrame({'enrollee_id':test["enrollee_id"],'target':predicted_final_knc})
prediction.to_csv('predicted1_knc_bal_standardscaler.csv',index=False)



C:\Users\umairansari\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [26]:
# final_features_to_be_considered_frc=feature_importance_plot_rev(training_x,rfc,.01)
# columns_considered=final_features_to_be_considered_frc["name_of_column"]
# print(columns_considered.values)


#stacking several models

# lrc=LogisticRegression(class_weight='balanced')
# pac=PassiveAggressiveClassifier(class_weight='balanced')
# rc=RidgeClassifierCV(class_weight='balanced')
# #sgc=SGDClassifier(class_weight='balanced')
# enc=ElasticNetCV()
# gnbc=GaussianNB()
# knc=KNeighborsClassifier()

level0=list()
level0.append(('lr',LogisticRegression()))
level0.append(('pac',PassiveAggressiveClassifier()))
level0.append(('rc',RidgeClassifierCV()))
level0.append(('gnbc',GaussianNB()))

level1 = LogisticRegression()
# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)

from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
# rfr=AdaBoostRegressor()
# rfr.fit(full_train_x,full_train_y)
# test_final=test.drop(["target","flag","enrollee_id"],axis=1)
# predicted_final_rfr=rfr.predict(test_final)
# #print(classification_report(predicted_final_lrc,validation_y))
# prediction=pd.DataFrame({'enrollee_id':test["enrollee_id"],'target':predicted_final_rfr})
# prediction.to_csv('predicted1_stacking_adar_bal_standardscaler.csv',index=False)

In [27]:
model.fit(full_train_x,full_train_y)
test_final=test.drop(["target","flag","enrollee_id"],axis=1)
predicted_final_stacking=model.predict(test_final)
#print(classification_report(predicted_final_lrc,validation_y))
prediction=pd.DataFrame({'enrollee_id':test["enrollee_id"],'target':predicted_final_stacking})
prediction.to_csv('predicted1_stacking2_bal_standardscaler.csv',index=False)

C:\Users\umairansari\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\umairansari\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#

In [28]:
training.corr()

,city,city_development_index,company_type,education_level,enrolled_university,enrollee_id,experience,gender,last_new_job,relevent_experience,...,engineered_exp_training,engineered_exp_training1,sum,min,max,mean,std,skew,kurt,med
city,1.000000,0.326516,0.008261,-0.002657,-0.005467,-0.011282,0.096489,-0.028777,0.058492,-0.018078,...,-0.046996,0.050736,0.029485,0.948033,0.000116,0.029513,-0.006551,0.017273,-0.020697,0.112493
city_development_index,0.326516,1.000000,-0.076021,0.037206,-0.123983,-0.033675,0.294039,-0.135284,0.175051,0.062381,...,-0.124718,0.152020,0.039980,0.397434,-0.005744,0.039961,-0.010256,-0.102248,-0.127670,0.181854
company_type,0.008261,-0.076021,1.000000,-0.194197,0.124255,0.036118,-0.141304,0.070387,-0.205868,-0.343095,...,0.101347,-0.060844,-0.007727,-0.040787,0.015105,-0.007726,0.017074,0.098780,0.097089,-0.173393
education_level,-0.002657,0.037206,-0.194197,1.000000,-0.092788,-0.017951,0.251671,-0.075039,0.203493,0.234385,...,-0.156325,0.138409,0.028095,0.046209,-0.006523,0.028094,-0.008948,-0.127144,-0.134116,0.165843
enrolled_university,-0.005467,-0.123983,0.124255,-0.092788,1.000000,0.018923,-0.264712,0.069155,-0.165547,-0.243483,...,0.139156,-0.133904,-0.014384,-0.025925,0.021277,-0.014377,0.023335,0.125051,0.134626,-0.159833
enrollee_id,-0.011282,-0.033675,0.036118,-0.017951,0.018923,1.000000,-0.028400,-0.037870,-0.024685,-0.026482,...,0.018136,-0.010444,-0.003393,-0.006930,0.000869,-0.003395,0.001160,0.026778,0.025768,-0.036435
experience,0.096489,0.294039,-0.141304,0.251671,-0.264712,-0.028400,1.000000,-0.123199,0.466230,0.316400,...,-0.376779,0.529505,0.140990,0.156551,0.013826,0.140985,0.008935,-0.430413,-0.475302,0.471674
gender,-0.028777,-0.135284,0.070387,-0.075039,0.069155,-0.037870,-0.123199,1.000000,-0.102991,-0.113295,...,0.063815,-0.065514,-0.015944,-0.066261,0.001745,-0.015942,0.003231,0.050692,0.055928,-0.084272
last_new_job,0.058492,0.175051,-0.205868,0.203493,-0.165547,-0.024685,0.466230,-0.102991,1.000000,0.236494,...,-0.214083,0.242299,0.082661,0.134875,0.000808,0.082657,-0.007864,-0.250592,-0.225901,0.970705
relevent_experience,-0.018078,0.062381,-0.343095,0.234385,-0.243483,-0.026482,0.316400,-0.113295,0.236494,1.000000,...,-0.194672,0.174792,0.052301,0.037885,0.008659,0.052299,0.005619,-0.157420,-0.166785,0.195557


In [29]:
lrc=LogisticRegression()
lrc.fit(full_train_x,full_train_y)

C:\Users\umairansari\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 335 tasks      | elapsed: 19.4min
[Parallel(n_jobs=-1)]: Done 1030 tasks      | elapsed: 34.7min
[Parallel(n_jobs=-1)]: Done 1920 out of 1920 | elapsed: 43.1min finished
C:\Users\umairansari\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                          dual=False, fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [1.0, 0.01], 'dual': [False, True],
                         'fit_intercept': [True, False],
                         'max_iter': [100, 1000],
                         'penalty': ['l2', 'l1', 'elasticnet'],
                         'random_state': [42, 71],
                         'solver': ['lbfgs', 'liblinear'],
 

In [36]:
lrc=LogisticRegression(C= 1.0,
  dual= True,
  fit_intercept= True,
  max_iter= 100,
  penalty='l2',
  random_state= 71,
  solver='liblinear',tol= 0.0001)

In [37]:
lrc.fit(full_train_x,full_train_y)
predicted_final_lrc_hyp=lrc.predict(test_final)
#print(classification_report(predicted_final_lrc,validation_y))
prediction=pd.DataFrame({'enrollee_id':test["enrollee_id"],'target':predicted_final_lrc_hyp})
prediction.to_csv('predicted1_lrc_bal_standardscaler_hyp.csv',index=False)

C:\Users\umairansari\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [34]:
gcv.best_estimator_,gcv.best_params_,gcv.best_score_

(LogisticRegression(C=1.0, class_weight='balanced', dual=True,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                    max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=71, solver='liblinear', tol=0.0001, verbose=0,
                    warm_start=False),
 {'C': 1.0,
  'dual': True,
  'fit_intercept': True,
  'max_iter': 100,
  'penalty': 'l2',
  'random_state': 71,
  'solver': 'liblinear',
  'tol': 0.0001},
 0.8661147194432723)

In [ ]:
param_dict={
            'eps':[0.001,.01], 
            'l1_ratio':[0.5,0.1],
            'max_iter':[100,1000], 
            'n_alphas':[100,500],
            'normalize':[True,False],
            'selection':['cyclic','random'], 
            'tol':[0.0001,.001]
}
gcv=RandomizedSearchCV(ElasticNetCV(),param_grid=param_dict,cv=2,n_jobs=-1,verbose=-2)
gcv.fit(full_train_x,full_train_y)
gcv.best_params_,gcv.best_score_